##  Creating and testing a Mean Reversion RSI_MA Long/Short Strategy in Crypto ##

In [1]:
# Import Modules
import vectorbt as vbt
import numpy as np 
import pandas as pd 
import datetime

In [2]:
# Get Data
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days = 7)

stocks = vbt.YFData.download(
    ["ADA-USD", "BNB-USD", "BTC-USD", "DOGE-USD", "ETH-USD", "SHIB-USD", "SOL-USD", "THETA-USD", "XRP-USD"],   
    interval = "1D", 
    missing_index = "drop").get("Close")

/Users/guilhermegomes/opt/anaconda3/lib/python3.9/site-packages/vectorbt/data/base.py:527: UserWarning: Symbols have mismatching index. Dropping missing data points.
  data = cls.align_index(data, missing=missing_index)


In [3]:
# Creating a Function 
def combine_rsi_ma(stocks, fast_window, slow_window, rsi_window, top, bottom): 
    rsi = vbt.RSI.run(stocks, window = rsi_window).rsi.to_numpy()
    fast = vbt.MA.run(stocks, window = fast_window).ma.to_numpy()
    slow = vbt.MA.run(stocks, window = slow_window).ma.to_numpy()
    indicator = np.where((fast > slow) & (rsi < top), 1, 0)
    indicator = np.where((fast < slow) & (rsi > bottom), -1, indicator)
    return indicator

# Creating Indicator RSI_MA
indicator_rsi_ma = vbt.IndicatorFactory(
    class_name = "Combination", 
    short_name = "RSI_MA",
    input_names = ["close"],
    param_names = ["fast_window", "slow_window", "rsi_window", "top", "bottom"],
    output_names = ["value"],  
).from_apply_func(combine_rsi_ma, fast_window = 180, slow_window = 240, rsi_window = 14, top = 80, bottom = 20)

# Optimizing RSI_MA
res = indicator_rsi_ma.run(
    stocks,
    fast_window = np.arange(50, 150, step = 10, dtype = int),
    slow_window = np.arange(100, 250, step = 10, dtype = int),
    rsi_window = np.arange(1, 40, step = 3, dtype = int),
    top = np.arange(50, 90, step = 10, dtype = int),
    bottom = np.arange(10, 40, step = 10, dtype = int),
    param_product = True
)

# Defining Entries/Exits - Short_entries/Short_Exits
entries = res.value_crossed_above(0)
exits = res.value_crossed_below(0)
short_entries = res.value_crossed_below(0)
short_exits = res.value_crossed_above(0)
pf = vbt.Portfolio.from_signals(stocks, entries, exits, short_entries, short_exits, fees = 0.001, slippage = 0.001)
max_return = pf.total_return().groupby("symbol").max()
max_return.vbt.barplot(xaxis_title = "Stocks", yaxis_title = "Total Return", height = 300, width = 1000)
    

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': '…